In [6]:
from collections import defaultdict
import itertools
import pickle

#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the incidence matrix
#output: the list of edges
def matrix_to_edges(M):
    H = IncidenceStructure(M)
    E=[]
    for element in H.blocks():
        E.append(tuple(element))
    return E


# Named function
def default_value():
    return None

# Create a dictionary
as_co_dict = defaultdict(default_value)
as_co_dict[((1,2,3),(1,2,3),(1,2,3))] = 3/8

# Save the dictionary to a file
with open('as_co_dict.pickle', 'wb') as file:
    pickle.dump(as_co_dict, file)

#input: the list of edges of a Veblen hypergraph V
#output: the flattening of V
def flat(V):
    L=[]
    for element in V:
        L.append(tuple(element))
    W=Set(L)
    return sorted(tuple(W))

# input: hypergraph incidence matrix M, vertices X edges
# output: list of all rootings as a list of nonzero (row,col) pairs of M
def rootings(M):
    R = M.nrows()
    if R == 0 or M.ncols() == 0:
        return [[]]
    rootinglist = []
    for rownum in range(R):
        #print(rownum,M.submatrix(rownum,1))
        if M[rownum][0] == 0:
            rownum += 1
        else:
            tempmx = M.submatrix(0,1)
            for colnum in range(1,M.ncols()):
                if M[:,colnum] == M[:,0]:
                    #print('colnum=',colnum)
                    #print(M)
                    for j in range(rownum):
                        #print(j,tempmx)
                        tempmx[j,colnum-1] = 0
            #print('rownum=',rownum)
            #print(tempmx)
            templist = rootings(tempmx)
            rootinglist += [[rownum]+L for L in templist]
    return rootinglist


# input: hypergraph incidence matrix M, vertices X edges
# output: list of all digraphs of Euler rootings
def euler_rootings_digraph(M):
    rootinglist = rootings(M)
    R = M.nrows()
    returnlist = []
    edges=matrix_to_edges(M)
    dit=dict([(a,0) for a in edges])
    for L in rootinglist:
        A = zero_matrix(R)
        c=1
        for rownum in range(R):
            dicct=copy(dit)
            for j in range(len(L)):
                if M[rownum,j] != 0:
                    if rownum != L[j]:
                        A[L[j],rownum] += 1
                    else:
                        dicct[edges[j]]+=1
            c*=multinomial(dicct.values())
        D = DiGraph(A)
        if D.is_eulerian():
#             show(D)
            returnlist.append((D,c))
    return returnlist

#in: Eulerian digraph D
#out: number of spanning rooted subtrees of D
def tau(D):
    L = D.laplacian_matrix()
    L1 = L.submatrix(1,1)
    tau = L1.determinant()
    return tau


#in: distinguishable Eulerian digraph D
#out: number of distinguishable Euler tours of D
def count_euler_tours(D):
    c=tau(D)
    for v in D.vertices():
        c *= factorial(D.out_degree(v)-1)
    return(c)

# Finds C_H = sum_{faithful M} C_M for H which are connected.  Begin by specifying parameters.
#input: tuple of edges of a possibly disconnected k-uniform hypergraph
#output: Associated coefficient of H
def as_co(E):
    # Load the dictionary from the file
    with open('as_co_dict.pickle', 'rb+') as file:
        loaded_dict = pickle.load(file)
        if loaded_dict[E] == None:
            # print("YES")
            M=edges_to_matrix(E)
            s=0
            for pair in euler_rootings_digraph(M):
                D=pair[0] 
                #c is the product of in-degrees
                c=1
                for v in D.vertices():
                    c*=D.out_degree(v)
                s+=pair[1]*tau(D)/c
                # show(D)
                # print(pair[1])
            loaded_dict[E] = s
            pickle.dump(as_co_dict, file)
            # print(loaded_dict)
        return loaded_dict[E]

#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#in: edge set E
#out: minimum number of vertices on which E form a connected hypergraph.
def count_vertices(E):
    Vertex_set = set({})
    for edge in E:
        for vertex in edge:
            Vertex_set.add(vertex)
    return len(Vertex_set)


#in: a list of edges
#out: True if the hypergraph does not have a cut-edge or a joint vertex
def has_no_cut_edge(edges):
    H = IncidenceStructure(edges)

    if any(not IncidenceStructure(H.ground_set(), set(edges) - {ed}).is_connected() for ed in edges):
        return False
    
    G =Graph()
    for edge in H.blocks():
        G.add_edges(Subsets(edge,2))
    if G.blocks_and_cut_vertices()[1] != []:
        return False

    return True

def spanning_determine(E,n):
    Vertex_set = set({})
    for edge in E:
        for vertex in edge:
            Vertex_set.add(vertex)
    if len(Vertex_set) != n:
        return False
    else: 
        return True

#in: m many edges of the host graph and infragraph as an m-vector, A incidence matrix of the host graph
#out: edge multiset of the infragraph
def convert_to_edges(edges,infra):
    m=len(edges)
    n=count_vertices(edges)
    A = matrix([[1 if point in edge else 0 for edge in edges] for point in range(n)])
    your_dict = { tuple([i for i in range(n) if A[i][j] != 0]) : infra[j] for j in range(m)}
    infra_edges = tuple([key for key, count in your_dict.items() for _ in range(count)])
    return infra_edges
    
#in: the edge set of a simple connected 3-uniform hypergraph H, k uniformity, d nonnegative integer
#out: a dictionary of indecomposable Veblen infragraphs of the hypergraph H on <=d edges, grouped up to isomorphism
def list_indecomposable_infra(edges,d,B,indecomposable_infras):                  
    m=len(edges)
    for ell in range(4,d+1):
        # print(ell,d)
        for S in Subsets(range(m),ell):
            edge_subset = [edges[j] for j in S]
            if has_no_cut_edge(edge_subset): 
                # eliminates the subhypergraphs with a cut-edge, as they can not contain any indecomposable infragraph
                S = list(S)    
                M_S = B[:,S]
                nullspace_S = M_S.right_kernel()
                for el in nullspace_S:
                    d_infra = sum([int(coordinate) for coordinate in el])
                    if prod(el) != 0 and d_infra <= d : 
                        #eliminates the infragraphs with zero copies of a certain edge, because they are already covered previously for smaller ell
                        infra = tuple([el[S.index(i)] if i in S else 0 for i in range(m)])
                        new_indecomposable = True
                        if any([ vect for t in range(4,d_infra-3) for vect in indecomposable_infras[t] if all(v1 <= v2 for v1, v2 in zip(vect, infra)) ]):
                            #checks if infra has any indecomposable infragraph vect as an edge-subgraph, which means infra is decomposable
                            new_indecomposable = False
                        if new_indecomposable:
                            indecomposable_infras[d_infra].append(infra)

    return indecomposable_infras

#in: the edge set of a simple connected 3-uniform hypergraph H, k uniformity, d nonnegative integer
#out: a dictionary of all Veblen infragraphs of the hypergraph H on <=d edges, grouped up to isomorphism
def list_all_infragraphs(edges,d):
    m=len(edges)
    n=count_vertices(edges)  
    F = GF(3)
    M = MatrixSpace(F,n,m)
    V = VectorSpace(QQ,m)

    B = M([[1 if point in edge else 0 for edge in edges] for point in range(n)])
    #incidence matrix with the order of the edges preserved
    
    three_basis = []
    g = [0]*m
    for i in range(m):
        h = deepcopy(g)
        h[i] = 3
        three_basis.append(tuple(h))
    
    indecomposable_infras = defaultdict(lambda: [] )
    
    for el in three_basis:
        indecomposable_infras[3].append(el)    
    
    L = list_indecomposable_infra(edges,d,B,indecomposable_infras)
    decomposable_infras = defaultdict(lambda: 0 )
    partitions = list( Partitions(d, min_part=3) )
    full_list = []
    for cell in partitions:
        partition_dict = cell.to_exp_dict()
        if all([L[d_part]!= [] for d_part in partition_dict]):
            candidates = {}
            for d_part in partition_dict:
                candidates[d_part] = [sum([coefficients[i] * V(L[d_part][i]) for i in range( len(L[d_part]) )] ) for coefficients in IntegerVectors(partition_dict[d_part], len( L[d_part] ) ) ] 
            combinations = itertools.product(*candidates.values())
            for combo in combinations:
                decomposable = tuple(sum(combo))
                if not decomposable in full_list:
                    full_list.append(decomposable)
                    decomposable_edges = convert_to_edges(edges,decomposable)
                    inc_1 = IncidenceStructure(decomposable_edges)
                    new_decomposable = True
                    for Veblen in decomposable_infras:
                        veblen_edges = convert_to_edges(edges,Veblen)
                        inc_2 = IncidenceStructure(veblen_edges)
                        if inc_1.is_isomorphic(inc_2):
                            new_decomposable = False
                            decomposable_infras[tuple(Veblen)]+=1
                            break
                    if new_decomposable:
                        decomposable_infras[tuple(decomposable)] = 1
    # print(full_list)
    return decomposable_infras

#in: edges of a simple hypergraph H as a list of tuples with vertices in [n] = {0,1,2,...,n-1}
#out: char_poly of the hypergraph H up to codegree lm
def char_poly_via_infra(edges,n,lm):
    m = len(edges)
    k = len(edges[0])
    degree = n * (k-1)^(n-1)
    if edges == []:
        return 1
    else:
        x, t=var("x t")
        log_chi = -m * (3/8)* (k-1)^n * x^3
        for d in range(k+1, lm+1):
            S_d = 0
            L = list_all_infragraphs(edges,d)
            for el, multiplicity in L.items():
                edges_infra = convert_to_edges(edges, el)
                inc = IncidenceStructure(edges_infra)
                if inc.is_connected():
                    S_d += as_co(edges_infra) * multiplicity
            log_chi += -(k-1)^n * S_d * x^d
            chi =  expand( t^degree * taylor( exp(log_chi) ,x, 0, d).subs(x == 1/t) )
            # if S_d != 0:
            #     print("d=",d,"chi= ",chi)
        return chi

    
edges_X = [ (1, 2, 3), (2, 3, 4), (1, 2, 0), (3, 4, 0)]
edges_Y = [ (1, 2, 3), (2, 3, 4), (2, 3, 0), (1, 4, 0)]
# (n,m) = (5,4) hypomorphic but not isomorphic
# differ at d=5

lm=9

n_X = count_vertices(edges_X)
print("edges_X's char_poly up to codegree",lm,"= ",char_poly_via_infra(edges_X, n_X, lm))

n_Y  = count_vertices(edges_Y)
print("edges_Y's char_poly up to codegree",lm,"= ",char_poly_via_infra(edges_Y, n_Y, lm))

/tmp/ipykernel_751/3362214803.py:128: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  for v in D.vertices():


edges_X's char_poly up to codegree 9 =  t^80 - 48*t^77 + 993*t^74 - 11896*t^71
edges_Y's char_poly up to codegree 9 =  t^80 - 48*t^77 - 54*t^75 + 993*t^74 - 216*t^73 + 2106*t^72 - 11968*t^71
